### GDP by Industry / Value Added

Covers basic broad categories. Still need to expand discussion to cover levels (per capita) and add text that covers both levels and changes. 

https://apps.bea.gov/iTable/iTable.cfm?ReqID=51&step=1

In the latest data, the quarter format is different (Q1 instead of I). I manually replaced the row containing the quarter. 

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrsrm = {'I': 1, 'II': 4, 'III': 7, 'IV': 10}

### Contribution to GDP

##### Historical Data

In [2]:
s = ['Quarter', 'Government', 'Utilities', 'Wholesale trade', 'Retail trade', 
     'Transportation and warehousing', 'Private goods-producing industries [1]',
     'Private services-producing industries [2]',
     'Finance, insurance, real estate, rental, and leasing',
     'Educational services, health care, and social assistance', 
     'Construction', 'Manufacturing', 'Information', 
     'Professional and business services']

d = {}
for dset in [1, 2]:
    # Adjust top and bottom line 1, 2 (downloaded data manually)
    df = pd.read_csv(f'raw/gdpbyindustry{dset}.csv', skiprows=4).T

    columns = df.iloc[1].str.strip()
    if dset == 3:
        columns[0] = 'Quarter'

    df.columns = columns

    si = [i for i in s if i in df.keys()]
    df = df.iloc[2:].loc[:, si]

    if dset == 3:
        dates = list(zip([dt[:4] for dt in df.index], df.Quarter.map(qtrsrm).values))
        date_index = pd.to_datetime([f'{year}-{month}-01' for year, month in dates])

        df = (df.reset_index().drop(['index', 'Quarter'], axis=1)
                .set_index(date_index).astype(float))

    else:
        dates = pd.to_datetime([f'{year}-01-01' for year in df.index])
        df = (df.reset_index().drop(['index'], axis=1)
                .set_index(dates).astype(float))    

    df.index.name = 'date'
    df['TTU'] = df[['Utilities', 'Wholesale trade', 'Retail trade', 
                    'Transportation and warehousing']].sum(axis=1)
    df['FIRE'] = df['Finance, insurance, real estate, rental, and leasing'] + df['Information']
    df['Oth_Serv'] = (df['Private services-producing industries [2]'] - df['TTU']
                      - df['FIRE'])
    df['GoodsTTU'] = df['Private goods-producing industries [1]'] + df['TTU']

    d[dset] = df

In [3]:
years = ','.join(map(str, range(2004, 2024)))

url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&DataSetName=GDPbyIndustry'+
       f'&Year={years}&Industry=ALL&tableID=13&Frequency=Q'+
        '&ResultFormat=json')

r = requests.get(url)

df = pd.DataFrame(json.loads(r.text)['BEAAPI']['Results'][0]['Data'])
df['Month'] = df.Quarter.map(qtrsrm)
df['Day'] = 1
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

data = df.groupby(['Date', 'IndustrYDescription']).DataValue.max().unstack().astype('float')
data['TTU'] = data[['Utilities', 'Wholesale trade', 'Retail trade', 
                'Transportation and warehousing']].sum(axis=1)
data['FIRE'] = data['Finance, insurance, real estate, rental, and leasing'] + data['Information']
data['Oth_Serv'] = (data['Private services-producing industries<sup>2</sup>'] - data['TTU']
                - data['FIRE'])
data['GoodsTTU'] = data['Private goods-producing industries<sup>1</sup>'] + data['TTU']

data = data.rename({'Private goods-producing industries<sup>1</sup>': 
                    'Private goods-producing industries [1]',
                    'Private services-producing industries<sup>2</sup>': 
                    'Private services-producing industries [2]'}, axis=1)

srs = ['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU', 
       'Educational services, health care, and social assistance', 
       'Construction', 'Manufacturing', 'Retail trade', 'Information',
       'Professional and business services']

srs1 = ['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU']
data1 = pd.concat([d[1], d[2]]).loc['1989':'2005'].resample('QS').ffill().iloc[:-1]
final = pd.concat([data1, data])
final.to_csv(data_dir / 'gdpvafull.csv', index_label='date', float_format='%g')
final[srs1].to_csv(data_dir / 'gdpva.csv', index_label='date', float_format='%g')

KeyError: 'Results'

In [ ]:
lt = final[srs1].iloc[-1]
pr = final[srs1].iloc[-2]
pcdt = '2019-10-01'
pc = final.loc[pcdt, srs1]
ltdate = dtxt(final.index[-1])['qtr1']
prdate = dtxt(final.index[-2])['qtr1']
pcdate = dtxt(pd.to_datetime(pcdt))['qtr1']

gttu = f"{lt['GoodsTTU']:.1f}"
prgttu = value_text(pr['GoodsTTU'], style='contribution_of', ptype='pp', threshold=0.1)
pcgttu = value_text(pc['GoodsTTU'], style='contribution_of', ptype='pp', threshold=0.1)

fire = value_text(lt['FIRE'], style='contribution', ptype='pp', threshold=0.1)
prfire = value_text(pr['FIRE'], style='contribution', ptype='pp', threshold=0.1)
pcfire = value_text(pc['FIRE'], style='contribution', ptype='pp', threshold=0.1)

oths = value_text(lt['Oth_Serv'], style='contribution_to', ptype='pp', threshold=0.1)
proths = value_text(pr['Oth_Serv'], style='contribution_of', ptype='pp', threshold=0.1)
pcoths = value_text(pc['Oth_Serv'], style='contribution_of', ptype='pp', threshold=0.1)

gov = value_text(lt['Government'], style='contribution', ptype='pp', threshold=0.1)
prgov = value_text(pr['Government'], style='contribution', ptype='pp', threshold=0.1)
pcgov = value_text(pc['Government'], style='contribution', ptype='pp', threshold=0.1)

In [ ]:
text = (f'In {ltdate}, the combined contribution to GDP growth from '+
        'private goods-producing industries and trade, transportation, '+
        f'and utilities is {gttu} percentage points, following {prgttu} '+
        f'in {prdate}, and compared to {pcgttu} in {pcdate}. The group '+
        'of private service-providing industries that include finance, '+
        'insurance, real estate, as well as the information industry, '+
        f'{fire} in {ltdate}, {prfire} in {prdate}, and {pcfire} '+
        f'in {pcdate}. \n\n Other private services-providing industries, '+
        f'which are wide-ranging and described above, {oths} real GDP '+
        f'growth in {ltdate}, '+
        f'following {proths} in {prdate}, and compared to {pcoths} in '+
        f'{pcdate}. Combined federal, state, and local government '+
        f'{gov} in {ltdate}, {prgov} the prior quarter, and {pcgov} '+
        f'in {pcdate}, prior to the pandemic.')
write_txt(text_dir / 'gdpva_growth.txt', text)
print(text)

### Nominal

In [ ]:
nom_ann = pd.read_csv('../data/GDPVA_nom.csv', skiprows=4).dropna().T.iloc[1:]
nom_ann.columns = nom_ann.iloc[0].str.strip()
nom_ann.columns.name = 'IndustrYDescription'
nom_ann = nom_ann.iloc[1:]
nom_ann.index = pd.to_datetime([f"{i}-07-01" for i in nom_ann.index])
nom_ann.index.name = 'Date'

# Duplicate columns dropped:
nom_ann = nom_ann.loc[:,~nom_ann.columns.duplicated()]

nom_ann = nom_ann.rename({'Private goods-producing industries1': 
                          'Private goods-producing industries [1]',
                          'Private services-producing industries2': 
                          'Private services-producing industries [2]'}, axis=1)

In [ ]:
years = ','.join(map(str, range(2005, 2024)))

url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&DataSetName=GDPbyIndustry'+
       f'&Year={years}&Industry=ALL&tableID=1&Frequency=Q'+
        '&ResultFormat=json')

r = requests.get(url)

df = pd.DataFrame(json.loads(r.text)['BEAAPI']['Results'][0]['Data'])
df['Month'] = df.Quarter.map(qtrsrm)
df['Day'] = 1
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

data = df.groupby(['Date', 'IndustrYDescription']).DataValue.max().unstack().astype('float')

data = data.rename({'Private goods-producing industries<sup>1</sup>': 
                    'Private goods-producing industries [1]',
                    'Private services-producing industries<sup>2</sup>': 
                    'Private services-producing industries [2]'}, axis=1)

data = pd.concat([nom_ann, data])

data['TTU'] = data[['Utilities', 'Wholesale trade', 'Retail trade', 
                'Transportation and warehousing']].sum(axis=1)
data['FIRE'] = data['Finance, insurance, real estate, rental, and leasing'] + data['Information']
data['Oth_Serv'] = (data['Private services-producing industries [2]'] - data['TTU']
                - data['FIRE'])
data['Goods'] = data['Private goods-producing industries [1]']
data['GoodsTTU'] = data['Private goods-producing industries [1]'] + data['TTU']

data.to_csv(data_dir / 'gdpva_nom.csv', index_label='date', float_format='%g')

### Real

In [ ]:
real_ann = pd.read_csv('../data/GDPVA_real.csv', skiprows=4).dropna().T.iloc[1:]
real_ann.columns = real_ann.iloc[0].str.strip()
real_ann.columns.name = 'IndustrYDescription'
real_ann = real_ann.iloc[1:]
real_ann.index = pd.to_datetime([f"{i}-07-01" for i in real_ann.index])
real_ann.index.name = 'Date'

# Duplicate columns dropped:
real_ann = real_ann.loc[:,~real_ann.columns.duplicated()]

real_ann = real_ann.rename({'Private goods-producing industries2': 
                            'Private goods-producing industries [1]',
                            'Private services-producing industries3': 
                            'Private services-producing industries [2]'}, axis=1)

In [ ]:
years = ','.join(map(str, range(2005, 2024)))

url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
        '&method=GetData&DataSetName=GDPbyIndustry'+
       f'&Year={years}&Industry=ALL&tableID=10&Frequency=Q'+
        '&ResultFormat=json')

r = requests.get(url)

df = pd.DataFrame(json.loads(r.text)['BEAAPI']['Results'][0]['Data'])
df['Month'] = df.Quarter.map(qtrsrm)
df['Day'] = 1
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

data = df.groupby(['Date', 'IndustrYDescription']).DataValue.max().unstack().astype('float')

data = data.rename({'Private goods-producing industries<sup>2</sup>': 
                    'Private goods-producing industries [1]',
                    'Private services-producing industries<sup>3</sup>': 
                    'Private services-producing industries [2]'}, axis=1)

data = pd.concat([real_ann, data])

data['TTU'] = data[['Utilities', 'Wholesale trade', 'Retail trade', 
                'Transportation and warehousing']].sum(axis=1)
data['FIRE'] = data['Finance, insurance, real estate, rental, and leasing'] + data['Information']
data['Oth_Serv'] = (data['Private services-producing industries [2]'] - data['TTU']
                - data['FIRE'])
data['Goods'] = data['Private goods-producing industries [1]']
data['GoodsTTU'] = data['Private goods-producing industries [1]'] + data['TTU']

data.to_csv(data_dir / 'gdpva_real.csv', index_label='date', float_format='%g')

In [ ]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC'] / 1_000

nom = pd.read_csv(data_dir / 'gdpva_nom.csv', parse_dates=['date'], index_col='date')
real = pd.read_csv(data_dir / 'gdpva_real.csv', parse_dates=['date'], index_col='date')

adj = ((real / real.iloc[-1]) * nom.iloc[-1]).div(pop, axis=0).dropna(how='all')

srs1 = ['Government', 'FIRE', 'GoodsTTU', 'Oth_Serv', 'Federal', 'State and local']
adj[srs1].to_csv(data_dir / 'gdpva_pc.csv', index_label='date', float_format='%g')

ltdt = dtxt(adj.index[-1])['qtr1']
write_txt(text_dir / 'gdpva_ltdt.txt', ltdt)

ltdate = dtxt(nom.index[-1])['qtr1']

sh = nom.iloc[-1].divide(nom['Gross domestic product'].iloc[-1], axis=0) * 100
ag = sh['Agriculture, forestry, fishing, and hunting']
mn = sh['Mining']
con = sh['Construction']
mfg = sh['Manufacturing']
ttu = sh['TTU']
fi = sh['Finance, insurance, real estate, rental, and leasing']
info = sh['Information']
pbs = sh['Professional and business services']
ehs = sh['Educational services, health care, and social assistance']
aer = sh['Arts, entertainment, recreation, accommodation, and food services']
gov = sh['Government']

text = ('The first category combines private goods producing industries: '+
        f'agriculture, forestry, fishing, and hunting ({ag:.1f} percent '+
        f'of GDP in {ltdate}); mining ({mn:.1f} percent of GDP); construction '+
        f'({con:.1f} percent); and manufacturing ({mfg:.1f} percent), with '+
        f'trade, transportation, and utilities (TTU, combined {ttu:.1f} '+
        'percent of GDP). The second category is finance, insurance, and '+
        f'real estate (FIRE, {fi:.1f} percent of GDP in {ltdate}) '+
        f'combined with the information industry ({info:.1f} percent of GDP), '+
        'labeled as FIRE+. \n\n The remaining private services-providing industries '+
        f'include: professional and business services ({pbs:.1f} percent of '+
        f'GDP in {ltdate}); education, health care, and social services '+
        f'({ehs:.1f} percent of GDP); and arts, entertainment, and recreation '+
        f'({aer:.1f} percent). Separately, public-sector value added in '+
        'production, at the federal, state, and local levels, is captured '+
        f'by the government category ({gov:.1f} percent of GDP).')
write_txt(text_dir / 'gdpva_sh.txt', text)
print(text)

In [ ]:
colors = {'GoodsTTU': 'purple!70!blue', 'FIRE': 'red!90!white', 
          'Oth_Serv': 'blue!90!white', 'Government': 'orange!80!white'}
cl = {k: c_line(color) for k, color in colors.items()}
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
aval = adj.multiply(1000).applymap('\${:,.0f}'.format)
alt = aval.iloc[-1]
apr = aval.loc[prdt]

text = (f'In {ltdate}, private goods-producing industries and the trade, '+
        f'transportation, and utilities industries add {alt.GoodsTTU} '+
        f'per person in domestic production, on an annualized basis, compared '+
        f'to {apr.GoodsTTU} in {prdate} {cl["GoodsTTU"]}. Private finance, '+
        'insurance, real estate, and information industry services '+
        f'add {alt.FIRE} in combined value, per capita in {ltdate} and '+
        f'{apr.FIRE} in {prdate} {cl["FIRE"]}. \n\n All other private '+
        'services-producing industries combined value added per person '+
        f'is {alt.Oth_Serv} in {ltdate} and {apr.Oth_Serv} in {prdate} '+
        f'{cl["Oth_Serv"]}. Government value added is {alt.Government} '+
        f'per person in {ltdate} and {apr.Government} in {prdate} '+
        f'{cl["Government"]}. ')
write_txt(text_dir / 'gdpva_pc.txt', text)
print(text)

In [ ]:
n1 = {'GoodsTTU': '\hspace{0.5mm} {\color{purple!70!blue}\\textbf{---}} Goods and TTU ',
      'Manufacturing': '\hspace{6mm} Manufacturing ',
      'Construction': '\hspace{6mm} Construction ',
      'Retail trade': '\hspace{6mm} Retail Trade ',
      'FIRE': '\hspace{0.5mm} {\color{red!90!white}\\textbf{---}} FIRE+ ',
      'Finance and insurance': '\hspace{6mm} Finance \& Insurance ',
      'Information': '\hspace{6mm} Information ',
      'Oth_Serv': '\hspace{0.5mm} {\color{blue!90!white}\\textbf{---}} Other Services ',
      'Educational services, health care, and social assistance': '\hspace{6mm} Education \& Healthcare ',
      'Professional and business services': '\hspace{6mm} Professional \& Business',
      'Government': '\hspace{0.5mm} {\color{orange!80!white}\\textbf{---}} Government '}

lt = adj[n1.keys()].rename(n1, axis=1).iloc[[-1, -2]]
pc = adj[n1.keys()].rename(n1, axis=1).loc[[prdt, '2005-01-01', '1997-07-01']]
tbl = pd.concat([lt, pc]).T * 1_000
tbl = tbl.applymap('{:,.0f}'.format)
tbl.iloc[0, 0] = f'\${tbl.iloc[0, 0]}'
tbl.columns = [dtxt(i)['qtr1'] for i in tbl.columns[:-1]] + ['1997 (A)']
tbl.to_csv(data_dir / 'gdpva_pc.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

### Government value added

In [ ]:
ltdt1 = dtxt(adj.index[-1])['qtr2']
ltdt = dtxt(adj.index[-1])['qtr1']

fltvaln = nom['Federal'].iloc[-1]
sltvaln = nom['State and local'].iloc[-1]
sprvaln = nom.loc[prdt, 'State and local']

fedcol = 'blue!70!white'
slgcol = 'red'
text = (f'In {ltdt1}, the federal government value added in domestic '+
        f'production is \${fltvaln:.1f} billion, equivalent to {alt.Federal} '+
        f'per capita {c_line(fedcol)}. In {prdate}, federal government added '+
        f'{apr.Federal} in value to domestic production, per capita, '+
        'after adjusting for inflation. \n\nState and local governments '+
        f'added \${sltvaln:,.1f} billion in production value in {ltdt} '+
        f'and \${sprvaln:,.1f} billion in {prdate}, equivalent after inflation '+
        f'to {alt["State and local"]} and {apr["State and local"]} per capita, '+
        f'respectively {c_line(slgcol)}.')
write_txt(text_dir / 'govva.txt', text)
print(text)